In [ ]:
import anndata as ad
import pandas as pd
import numpy as np
import scanpy as sc
import decoupler as dc
import os
import re
import statistics
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
def validate_input_arguments (arguments_list):
    if arguments_list["out_path"] is None:
        print("Please provide an output path")
    elif arguments_list["out_path"][-1] != "/":
        arguments_list["out_path"] = arguments_list["out_path"] + "/"

    if arguments_list["celltype"] is None:
        print("Please provide the name of the metadata field containing cell type annotations")

    if arguments_list["condition"] is None:
        print("Please provide the name of the metadata field containing condition annotations")

    if arguments_list["organism"] is None:
        arguments_list["organism"] = "human"

    if arguments_list["comparison_list"] is None:
        arguments_list["comparison_list"] = np.nan

    if arguments_list["logfc"] is None:
        arguments_list["logfc"] = 0.0

    if arguments_list ["pval"] is None:
        arguments_list["pval"] = 0.05

    if arguments_list["reg"] is None:
        arguments_list["reg"] = load_dorothea_regulon(arguments_list["organism"])

    elif isinstance(arguments_list["reg"], str):
        arguments_list["reg"] = pd.read_csv(arguments_list["reg"], index_col=0)
        arguments_list["reg"] = pd.DataFrame.rename(arguments_list["reg"], columns={"source" : "tf"})

    if not "tf" in arguments_list["reg"] and "target" in arguments_list["reg"] and "weight"in arguments_list["reg"]:
        raise Exception("Not all necessary columns found in regulon table! Please make sure that the regulon has the columns source, target and weight!")
    
 
   
    return(arguments_list)



In [ ]:
class TFObj:
    def __init__(self,
    tf_activities_condition : list,
    tf_activities_cluster : list,
    average_gene_expression : list,
    regulon : pd.DataFrame,
    CTR_input_condition : list,
    CTR_input_cluster : list,
    intracellular_network_condition : list,
    intracellular_network_cluster : list):

        self.tf_activities_condition = tf_activities_condition
        self.tf_activities_cluster = tf_activities_cluster
        self.average_gene_expression = average_gene_expression
        self.regulon = regulon
        self.CTR_input_condition = CTR_input_condition
        self.CTR_input_cluster = CTR_input_cluster
        self.intracellular_network_condition = intracellular_network_condition
        self.intracellular_network_cluster = intracellular_network_cluster

def make_TFOBj(tf_activities_condition : list,
    tf_activities_cluster : list,
    average_gene_expression : list,
    regulon : pd.DataFrame,
    CTR_input_condition : list,
    CTR_input_cluster : list,
    intracellular_network_condition : list,
    intracellular_network_cluster : list):

        tf = TFObj(tf_activities_condition,
            tf_activities_cluster,
            average_gene_expression,
            regulon,
            CTR_input_condition,
            CTR_input_cluster,
            intracellular_network_condition,
            intracellular_network_cluster)

        return tf

In [ ]:
def AverageExpression(sub_object, celltype = None, name_iterable = None, outpath = None):
    gene_ids = sub_object.var.index.values
    #cluster line even necessary if not returned?
    #clusters = anndataobject.obs[celltype].cat.categories
    obs = sub_object[:,gene_ids].X.toarray()
    obs = np.expm1(obs)
    avg_df = pd.DataFrame(obs,columns=gene_ids,index= sub_object.obs[celltype])
    avg_df = avg_df.groupby(level=0, observed=False).mean()
    avg_df.T.to_csv(outpath + name_iterable + "_average_gene_expression_by_cluster_exp.csv")

    return avg_df.T


In [ ]:
def eval_pval(p_val):
    p_val = float(p_val)
    if p_val < 0.001: 
      txt = "***"
    elif p_val < 0.01: 
      txt = "**"
    elif p_val < 0.05: 
      txt = "*"
    else:
      txt = "ns"
    return(txt)


def eval_log_fc_tag(log_fc):
    if log_fc >= 1.0: 
      txt = "***"
    elif log_fc > 0.5: 
      txt = "**"
    elif log_fc > 0.0: 
      txt = "*"
    else:
      txt = "ns"
    return(txt)

In [ ]:
def create_unfiltered_tf_scores(tf_scores_df, condition, celltype, out_path):   
    summarized_tf_scores_df = tf_scores_df.groupby(celltype, observed = True).mean().T
    #tf_scores_df.groupby(celltype, observed = True).apply(display)
    #agg(["mean", "var"])
    summarized_tf_scores_df.to_csv(out_path + "/unfiltered_tf_scores_" + condition + ".csv")
    return summarized_tf_scores_df

#np.var() returns a value that is different from R's var(), whereas statistics.variance() is the same as R's var()
def save_variable_tf_score(filtered_summarized_tf_scores_df, condition, out_path, plot):
    filtered_summarized_tf_scores_df["var"] = filtered_summarized_tf_scores_df.apply(statistics.variance, axis=1).unique()
    filtered_summarized_tf_scores_df.to_csv(out_path + "/variable_tf_scores_" + condition + ".csv")

    if plot:
        top_variable_tfs = filtered_summarized_tf_scores_df.sort_values("var", ascending=False).head(n=20).drop(columns="var")
        fig, ax = plt.subplots(figsize=(8,7))   
        ax = sns.heatmap(top_variable_tfs, cmap="vlag", center=0, vmin=top_variable_tfs.min(axis=None), cbar_kws={"label": "z-score"})
        ax.set(xlabel="Cell Type", ylabel="Transcription Factor")
        ax.get_figure()
        plt.savefig(out_path + "/tf_activity_top20_variable_" + condition + ".pdf")
        plt.close()

    filtered_summarized_tf_scores_df_var = filtered_summarized_tf_scores_df
    return filtered_summarized_tf_scores_df_var
    

In [ ]:
def plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping, condition, out_path):
    filtered_summarized_tf_scores_df = filtered_summarized_tf_scores_df.drop(columns="var")
    sns.clustermap(filtered_summarized_tf_scores_df, cbar_kws={"label": "z-score"}, figsize=(8,7), cmap="vlag", center=0, yticklabels=False)
    plt.savefig(out_path + "/tf_activity_compressed_" + condition + ".pdf")
    plt.close()

    sns.clustermap(filtered_summarized_tf_scores_df, cbar_kws={"label": "z-score"}, figsize=(8,50), cmap="vlag", center=0, annot= tag_mapping, fmt="")
    plt.savefig(out_path + "/tf_activity_" + condition + ".pdf")
    plt.close()


    

In [ ]:
def map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers):
    genes = []
    clusters = []
    z_scores = []  

    anndataobject_markers = anndataobject_markers.set_index("gene")
    #print(filtered_summarized_tf_scores_df)
    
    for i in range(len(anndataobject_markers.index)):
        a = anndataobject_markers.index[i]
        for j in range(len(filtered_summarized_tf_scores_df.index)):
            b = filtered_summarized_tf_scores_df.index[j]
            if a == b:
                c = anndataobject_markers.columns.get_loc("cluster")
                cluster = anndataobject_markers.iloc[i, c]
                gene_rows = filtered_summarized_tf_scores_df.iloc[j]
                if isinstance(gene_rows, pd.Series):
                    gene_rows = gene_rows.to_frame().T

                for _, gene_row in gene_rows.iterrows():
                     score = gene_row[cluster]
                     genes.append(a)
                     clusters.append(cluster)
                     z_scores.append(score)

    z_score_df = pd.DataFrame({
        'gene': genes,
        'cluster': clusters,
        'z_score': z_scores
    })

    return z_score_df

In [ ]:
def get_significant_tfs_single(tf_activities_sub, celltype, condition, out_path, pval, logfc, name, plot):
    
    #does not work if condition is None
    renamed_condition = condition.replace(",", "_")
    name = name.replace(",", "_")

    single_result_path = out_path + renamed_condition 
    if not os.path.isdir(single_result_path):
        os.mkdir(single_result_path)

    tf_activities_scaled = sc.pp.scale(tf_activities_sub, copy= True)

    #tf_activities_sub = sc.pp.normalize_total(tf_activities_scaled, copy=True)
    #tf_activities_sub = sc.pp.log1p(tf_activities_sub, copy=True) 
    # "warning: seems to be already log transformed"

    number_of_clusters = len(tf_activities_sub.obs["new_annotation"].cat.categories) 


    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="wilcoxon", key_added="wilcoxon_markers", corr_method= "bonferroni")
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="wilcoxon_markers", key_added= "wilcoxon_markers_filtered")
    

    sc.tl.rank_genes_groups(tf_activities_sub, groupby= "new_annotation", reference="rest", method="t-test_overestim_var", key_added="t_test_overestim_var_markers", corr_method= "bonferroni" )
    #sc.tl.filter_rank_genes_groups(anndataobject, min_in_group_fraction=0, key="t_test_overestim_var_markers", key_added="t_test_overestim_filtered")


    result1 = tf_activities_sub.uns['wilcoxon_markers']
    groups = result1['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result1[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    result2 = tf_activities_sub.uns['t_test_overestim_var_markers']
    groups = result2['names'].dtype.names
    anndataobject_markers = pd.DataFrame(
    {group + '_' + key[:1]: result2[key][group]
    for group in groups for key in ['names','logfoldchanges','pvals','pvals_adj']})

    anndataobject_markers_wilcoxon = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="wilcoxon_markers")
    
    anndataobject_markers_wilcoxon.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)
    
    #documentation says this won't update the main dataframe but will only overwrite a copy in future pandas versions (after v3)
    anndataobject_markers_wilcoxon["tag"] = None
    anndataobject_markers_wilcoxon["log_fc_tag"] = None
    for i in range(len(anndataobject_markers_wilcoxon["pvals_adj"])):
         anndataobject_markers_wilcoxon["tag"].iloc[i,] = eval_pval(anndataobject_markers_wilcoxon["pvals_adj"].iloc[i,])
         anndataobject_markers_wilcoxon["log_fc_tag"].iloc[i,] = eval_log_fc_tag(anndataobject_markers_wilcoxon["logfoldchanges"].iloc[i,])
     

    anndataobject_markers_wilcoxon.to_csv(single_result_path + "/" + name + "_specific_markers_wilcoxon_test.csv",index=0)

    anndataobject_markers_t_over = sc.get.rank_genes_groups_df(tf_activities_sub, group = None, log2fc_min=0, key="t_test_overestim_var_markers")
 
    anndataobject_markers_t_over.rename(columns={"names":"gene", "group": "cluster"}, inplace=True)

    anndataobject_markers_t_over["tag"] = None
    anndataobject_markers_t_over["log_fc_tag"] = None
    for i in range(len(anndataobject_markers_t_over["pvals_adj"])):
         anndataobject_markers_t_over["tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["pvals_adj"].iloc[i,])
         anndataobject_markers_t_over["log_fc_tag"].iloc[i,] = eval_pval(anndataobject_markers_t_over["logfoldchanges"].iloc[i,])

    anndataobject_markers_t_over.to_csv(single_result_path + "/" + name + "_specific_markers_t_test_overestim_test.csv",index=0)

   #tag mapping wilcoxon
    tag_mapping_wilcox = anndataobject_markers_wilcoxon[["gene", "tag", "log_fc_tag", "cluster", "pvals_adj", "logfoldchanges"]]
    tag_mapping_wilcox = tag_mapping_wilcox[(tag_mapping_wilcox["pvals_adj"] < float(pval)) & 
                              ((tag_mapping_wilcox["logfoldchanges"] > float(logfc)) | 
                              (tag_mapping_wilcox["logfoldchanges"] < -float(logfc)))]

    tag_mapping_wilcox = tag_mapping_wilcox.pivot(index="gene", columns="cluster", values="tag")
    clusters = anndataobject_markers_wilcoxon["cluster"].unique()

    for cluster in clusters:
        if cluster not in tag_mapping_wilcox.columns:
            tag_mapping_wilcox[cluster] = np.nan

    tag_mapping_wilcox.fillna("ns", inplace=True)

    tf_activities_scaled.obs_names = tf_activities_scaled.obs[celltype]
    tf_scores_df = tf_activities_scaled.to_df()
    tf_scores_df.columns.name = None
    unfiltered_tf_scores = create_unfiltered_tf_scores(tf_scores_df, condition, celltype, single_result_path)
   
    #Filter to only include tfs that match the tag_mapping/are markers
    col_num = tf_scores_df.columns.isin(tag_mapping_wilcox.index)  
    filtered_tf_scores_df = tf_scores_df.loc[:, col_num] 
    
    filtered_summarized_tf_scores_df = filtered_tf_scores_df.groupby(celltype, observed = False).mean().T
    filtered_summarized_tf_scores_df.index.name = "gene"
    filtered_summarized_tf_scores_df.to_csv(f'{single_result_path}/tf_scores_{condition}.csv')
    tf_scores_variable_table = save_variable_tf_score(filtered_summarized_tf_scores_df, condition, single_result_path, plot)

    if plot:
        plot_tf_activity(filtered_summarized_tf_scores_df, tag_mapping_wilcox, condition, single_result_path)
    
    #filtered_summarized_tf_scores_df.index = re.sub((".,"), "_", filtered_summarized_tf_scores_df.index)
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_wilcoxon),  on=['gene', 'cluster'], how='inner')
    anndataobject_markers_wilcoxon_merged = anndataobject_markers_wilcoxon_merged[anndataobject_markers_wilcoxon_merged.tag != "ns"]
    anndataobject_markers_wilcoxon_merged.dropna(inplace=True)
    res_wilcoxon = anndataobject_markers_wilcoxon_merged[["gene","tag", "cluster", "z_score"]]

    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over.merge(map_z_value_filtered(filtered_summarized_tf_scores_df, anndataobject_markers_t_over),  on=['gene', 'cluster'], how='inner')
    #anndataobject_markers_t_over_merged = anndataobject_markers_t_over_merged[anndataobject_markers_t_over_merged.tag != "ns"]
    #anndataobject_markers_t_over_merged.dropna(inplace=True)
    #res_t_test = anndataobject_markers_t_over[["gene","tag", "cluster", "z_score"]]

    res_wilcoxon.to_csv(single_result_path + "/" + name + "_significant_markers_wilcoxon.csv", index=0)
    #res_t_test.to_csv(single_result_path + "/" + name + "_significant_markers_t_test.csv", index=0)

     #//TODO: 
    #delete one variant and put test type as a variable

    res = {}
    res["cluster"] = res_wilcoxon

    return res

In [ ]:
#check if it's okay to not pre set the column type as string or float (maybe no problem in python but was problem in r)

def create_empty_CTR_dataframe():
  
  empty_df = pd.DataFrame(
    columns=[
    "source",
    "target",
    "gene_A",
    "gene_B",
    "type_gene_A",
    "type_gene_B",
    "MeanLR"
    ])
  
  return(empty_df)


def add_entry_to_CTR_dataframe(source, target, gene_A, gene_B, type_gene_A, type_gene_B, MeanLR, row):
  df = pd.DataFrame(
      {"source" : source,
      "target" : target,
      "gene_A" : gene_A,
      "gene_B" : gene_B,
      "type_gene_A" : type_gene_A,
      "type_gene_B" : type_gene_B,
      "MeanLR" : MeanLR}, index=[row])

  return df

In [ ]:
def generate_CrossTalkeR_input(tf_activities, gene_expression, out_path, regulon = None):

  ligands = pd.read_csv("ligands_human.csv")
  R2TF = pd.read_csv("rtf_db.csv")

  sorted_regulon = regulon[["tf", "target"]]
  #sorted_regulon.rename(columns={"target" : "targets"})
  sorted_regulon = sorted_regulon.set_index("tf")

  R2TF = R2TF.set_index("tf")
  output_df = create_empty_CTR_dataframe()
  tf_l = create_empty_CTR_dataframe()
  r_tf = create_empty_CTR_dataframe()

  for row in range(len((tf_activities))):

    #if (tf_activities["z_score"].iloc[row] > 0):
    tf = str(tf_activities["gene"].iloc[row])
    if tf in sorted_regulon.index:
      targets = sorted_regulon.loc[tf, "target"]

    if tf in R2TF.index:
        receptors = R2TF.loc[tf, "receptor"]

    else:
        receptors = []
    tf_ligands = np.intersect1d(targets, ligands)

    if len(tf_ligands) > 0:
      for ligand in tf_ligands:
          expressed = False
          if ligand in gene_expression.index:
            ex_value = gene_expression.loc[ligand, tf_activities.iloc[row, 2]]
            if (ex_value != 0):
              expressed = True
          
          df_list_l = []
            
          if (expressed == True):
            df_list_l = add_entry_to_CTR_dataframe(source = tf_activities.iloc[row, 2],
                                                    target = tf_activities.iloc[row, 2],
                                                    gene_A =tf_activities.iloc[row, 0],
                                                    gene_B = ligand,
                                                    type_gene_A = "Transcription_Factor",
                                                    type_gene_B= "Ligand",
                                                    MeanLR= tf_activities.iloc[row, 3],
                                                    row= row)
              
            tf_l = pd.concat([tf_l, df_list_l], ignore_index=True)
            
    
      if (len(receptors) > 0):
        for receptor in receptors:
          df_list_r = [] 
          df_list_r = add_entry_to_CTR_dataframe(tf_activities.iloc[row, 2],
                                              tf_activities.iloc[row, 2],
                                              receptor,
                                              tf_activities.iloc[row, 0],
                                              "Receptor",
                                              "Transcription Factor",
                                              tf_activities.iloc[row, 3],
                                              row)
              
          print(df_list_r)
            
          r_tf = pd.concat([r_tf, df_list_r], ignore_index=True)

      #tf_l.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_l.csv", index=0)
      #r_tf.to_csv(single_result_path + "/" + renamed_condition + "_ctr_test_r.csv", index=0)

  #    r_tf["gene_A"] <- gsub("_", "+", r_tf$gene_A, fixed = TRUE)
   #   r_tf["gene_B"] <- gsub("_", "+", r_tf$gene_B, fixed = TRUE)
   #   tf_l["gene_A"] <- gsub("_", "+", tf_l$gene_A, fixed = TRUE)
   #   tf_l["gene_B"] <- gsub("_", "+", tf_l$gene_B, fixed = TRUE)

  output_df = pd.concat([output_df, r_tf], ignore_index=True)
  output_df = pd.concat([output_df, tf_l], ignore_index=True)
  #output_df.to_csv("tf_l_r.csv", index=0)


  return output_df

In [ ]:
def generate_intracellular_network(tf_activity_scores, gene_expression_list, outpath, regulon, organism = "human"):

    

In [ ]:
#ignore extra tfs from decoupler while writing script 

def tf_activity_analysis (anndataobject, arguments_list = None):
    
    if (isinstance(anndataobject, str)):
        anndataobject = ad.read_h5ad(anndataobject)

    arguments_list = validate_input_arguments(arguments_list)

    if not os.path.isdir(arguments_list["out_path"]):
        os.mkdir(arguments_list["out_path"])
        tf_path = arguments_list["out_path"] + "TF_results/"
        os.mkdir(tf_path)
    else:
        tf_path = arguments_list["out_path"] + "TF_results/"

    #rename the arguments inserted into argument list (eg protocol to condition)(unfinished)
    condition = anndataobject.obs[arguments_list["condition"]]

    #checks for tf activity csv, if nothing there, runs decoupler
    if isinstance(arguments_list["tf_activities"], str):
         tf_activities = ad.read_csv(arguments_list["tf_activities"])
         #print(tf_activities.obs.index)
         tf_activities.obs = anndataobject.obs.reindex(tf_activities.obs.index)
         tf_activities.obsm = anndataobject.obsm
         tf_activities.uns = anndataobject.uns
    elif arguments_list["tf_activities"] is None:
         dc.run_ulm(mat = anndataobject, net = (arguments_list["reg"]), source ='tf', target ='target', weight ='weight', verbose = True, use_raw = False)
         tf_activities = anndataobject.obsm['ulm_estimate']
         tf_activities.to_csv("decoupler_results_test.csv")

    #sets the stage for decision if single condition or comparison analysis is done

    if not np.isnan(arguments_list["comparison_list"]):
        if len(arguments_list["comparison_list"]) > 0 & len(anndataobject.obs["comparison_list"]) < 2:
            arguments_list["comparison_list"] <- np.nan
            print("Only one condition was found in the data, although a list of comparisons was provided. The analyses are performed only for the present condition!")

    #code for single condition  analysis

    if np.isnan(arguments_list["comparison_list"]):

        result_list = {}
        gene_expression_list = {}
        CTR_cluster_list = {}
        intranet_cluster_list = {}

        #creates loop until after tf activity score
        
        for name_iterable in anndataobject.obs.protocol.unique():
            sub_object = anndataobject[anndataobject.obs.protocol == name_iterable]
            tf_activities_sub = tf_activities[tf_activities.obs.protocol == name_iterable]

            sub_object_avg = AverageExpression(sub_object, name_iterable= name_iterable, celltype = arguments_list["celltype"], outpath= arguments_list["out_path"])
            
            #check how sub object is returned by average expression (subobject.T) or rename return?
            tf_activity_scores = get_significant_tfs_single(tf_activities_sub, celltype = arguments_list["celltype"],condition = name_iterable, out_path= tf_path, pval = arguments_list["pval"], logfc = arguments_list["logfc"], name = name_iterable, plot = arguments_list["plot"])

            #result_condition_list[[name_iterable]] = tf_activity_scores[]
            result_list[name_iterable] = tf_activity_scores
            #turn them into dictionaries?
            
            gene_expression_list[name_iterable] = sub_object_avg
            
            if (arguments_list["organism"] == "human"):
            
                CTR_cluster_list[name_iterable] = generate_CrossTalkeR_input(tf_activity_scores["cluster"],
                                                                            gene_expression_list[name_iterable],
                                                                            arguments_list["out_path"],                                             
                                                                            arguments_list["reg"])
    
            #else:
   
            #    CTR_cluster_list[[name_iterable]] = generate_CrossTalkeR_input_mouse(tf_activity_scores["cluster"],
            #                                                                        gene_expression_list[name_iterable]
            #                                                                        arguments_list["out_path"],
            #                                                                        arguments_list["reg"])
              
            intranet_cluster_list[name_iterable] = generate_intracellular_network(tf_activity_scores["cluster"],
                                                                                  gene_expression_list[name_iterable],
                                                                                  arguments_list["out_path"],
                                                                                  arguments_list["reg"],
                                                                                  arguments_list["organism"])
            
            tf = make_TFOBj(
                tf_activities_condition = list(),
                tf_activities_cluster = result_list,
                average_gene_expression = gene_expression_list,
                regulon = arguments_list["reg"],
                CTR_input_condition = list(),
                CTR_input_cluster = CTR_cluster_list,
                intracellular_network_condition = list(),
                intracellular_network_cluster = intranet_cluster_list)

        return tf

In [ ]:
result = tf_activity_analysis(anndataobject= "LR2TF_test_run/anndata_object.h5ad", arguments_list= {"out_path" : "script_test", "celltype" : "new_annotation", "condition" : "protocol", "organism" : None, "comparison_list" : None, "logfc" : "0", "pval" : None, "reg" : "filterd_regulon.csv", "tf_activities" : "decoupler_results.csv", "plot" : True})

In [ ]:
def add_node_type(df):
  df = mutate(gene_A = ifelse(type_gene_A == "Ligand", paste0(gene_A, "|L"), gene_A))

  df = mutate(gene_A = ifelse(type_gene_A == "Receptor", paste0(gene_A, "|R"), gene_A))

  df = mutate(gene_A = ifelse(type_gene_A == "Transcription Factor", paste0(gene_A, "|TF"), gene_A))
  
  df = mutate(gene_B = ifelse(type_gene_B == "Ligand", paste0(gene_B, "|L"), gene_B))
  df <- df %>%
    mutate(gene_B = ifelse(type_gene_B == "Receptor", paste0(gene_B, "|R"), gene_B))
  df <- df %>%
    mutate(gene_B = ifelse(type_gene_B == "Transcription Factor", paste0(gene_B, "|TF"), gene_B))


In [ ]:
def combine_LR_and_TF(tf_table, LR_prediction, out_path, condition, add_node_type = False):

  if isinstance(LR_prediction, pd.DataFrame):
    lr_table = LR_prediction
  else: 
    #fix this
    lr_table = read.csv(LR_prediction)
    lr_table.rownames = lr_table.columns["X"]
    lr_table.columns["X"] = None

  lr_ligands = np.unique(lr_table["gene_A"])
  lr_receptors = np.unique(lr_table["gene_B"])
  
  tf_receptor_interactions =  tf_table[tf_table['gene_A'].isin(lr_receptors)]
  tf_ligand_interactions = tf_table[tf_table["gene_B"].isin(lr_ligands)]

  complete_interactions = pd.concat([tf_receptor_interactions, tf_ligand_interactions])
  complete_interactions = pd.concat([complete_interactions, lr_table])
  if (add_node_type):
    complete_interactions = add_node_type(complete_interactions)
    
  complete_interactions.to_csv((out_path + "CrossTalkeR_input_" + condition + ".csv"), index=0)
  return(complete_interactions)

In [ ]:
#LINUX
table_ctr = pd.read_csv("/home/larissa/Documents/LR2TF_HiWi/LR2TF_test_run/CTR_LR.csv")
table_exp = pd.read_csv("/home/larissa/Documents/LR2TF_HiWi/LR2TF_test_run/EXP_LR.csv")

ctr_input = combine_LR_and_TF(result.CTR_input_cluster["control"], table_ctr, "script_test/", "control")
exp_input = combine_LR_and_TF(result.CTR_input_cluster["PMF,MF2"], table_exp, "script_test/", "PMF_MF2")

In [ ]:
#WINDOWS
table_ctr = pd.read_csv("LR2TF_test_run\\CTR_LR.csv")
table_exp = pd.read_csv("LR2TF_test_run\\EXP_LR.csv")

ctr_input = combine_LR_and_TF(result.CTR_input_cluster["control"], table_ctr, "script_test/", "control")
exp_input = combine_LR_and_TF(result.CTR_input_cluster["PMF,MF2"], table_exp, "script_test/", "PMF_MF2")